In [3]:
from gurobipy import *

M = Model()
# number of players in the team
n = 15
# player strength list, either 2.5 or 3.0 for each player - TODO: get actual player strengths
N = [2.5, 3.0, 2.5, 3.0, 2.5, 3.0, 3.0, 2.5, 2.5, 3.0, 3.0, 3.0, 3.0, 3.0, 2.5]
# opponent strength list, either 2.5 or 3.0 for each court position
N0 = [3.0, 3.0, 3.0, 3.0, 3.0, 2.5, 3.0, 3.0]
# number of games in the tournament
m = 8
# number of slots for a player to play in per game
o = 8
# minimum number of games each player must play
p = 2
# availability matrix. 1 if the ith player can make the jth game, 0 if they cannot
A = [[0 for j in range(m)] for i in range(n)]
A[3][0] = 1
A[12][0] = 1
A[5][0] = 1
A[10][0] = 1
A[2][0] = 1
A[4][0] = 1
A[13][0] = 1
A[6][0] = 1

A[12][1] = 1
A[5][1] = 1
A[8][1] = 1
A[11][1] = 1
A[10][1] = 1
A[1][1] = 1
A[13][1] = 1
A[14][1] = 1

A[9][2] = 1
A[14][2] = 1
A[8][2] = 1
A[5][2] = 1
A[0][2] = 1
A[7][2] = 1
A[1][2] = 1
A[13][2] = 1

A[1][3] = 1
A[9][3] = 1
A[12][3] = 1
A[14][3] = 1
A[4][3] = 1
A[3][3] = 1
A[13][3] = 1
A[6][3] = 1

A[14][4] = 1
A[12][4] = 1
A[9][4] = 1
A[11][4] = 1
A[1][4] = 1
A[4][4] = 1
A[13][4] = 1
A[6][4] = 1

A[14][5] = 1
A[12][5] = 1
A[1][5] = 1
A[5][5] = 1
A[2][5] = 1
A[7][5] = 1
A[13][5] = 1
A[6][5] = 1

A[3][6] = 1
A[12][6] = 1
A[1][6] = 1
A[9][6] = 1
A[0][6] = 1
A[10][6] = 1
A[13][6] = 1
A[6][6] = 1

A[1][7] = 1
A[5][7] = 1
A[11][7] = 1
A[2][7] = 1
A[4][7] = 1
A[13][7] = 1
A[6][7] = 1
# preference matrix. 1 if the ith player prefers the jth game and kth slot, -1 if they prefer not to, 0 if no preference
P = [[[0 for k in range(o)] for j in range(m)] for i in range(n)] 
for j in range(0,m):
    P[6][j][6] = 1 
    P[6][j][7] = 1 
    P[13][j][6] = 1
    P[13][j][7] = 1
# constant terms for each objective optimization, in order of: balanced games, spread schedules, high win %, player preferences,
# and heavily unbalanced (2.5/2.5 vs 3.0/3.0) matches
C = [2,.5,.5,1, 10]
# 1 if a player i plays in game j on the kth court, 0 if they do not
x = M.addVars(n, m, o, vtype=GRB.BINARY)
# 0 if the kth court for the jth game is balanced or favors the Romanian team, 1 if it disfavors the Romanian team
bal = M.addVars(m,5, vtype=GRB.BINARY)
# 0 if the kth court for the jth game is balanced or favors the Romanian team, 1 if it heavily disfavors the Romanian team
bal2 = M.addVars(m,5, vtype=GRB.BINARY)
# 0 if the ith player does not play consecutive matches starting with the jth game. 1 if they do
inds = M.addVars(n,m, vtype=GRB.BINARY)

M.update()

for i in range(0,n):
    M.addConstrs(x.sum(i, j, '*') <= A[i][j] for j in range(0,m)) #players can only play one court in a match, 
                                                                  #or none if they are not available
    M.addConstr(x.sum(i, '*', '*'), GRB.GREATER_EQUAL, p) #players must play at least p games in the season
    for j in range(0,m-1):
        M.addConstr(x.sum(i, j, '*') + x.sum(i,j+1, '*') - inds[i,j] <= 1) #ensures that inds[i,j] will be 1 if consecutive games

for j in range(0,m):
    for k in [0,2,4]:
        expr = LinExpr(-1*N0[k]-N0[k+1])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
            expr.add(x[i,j,k+1], N[i])
        expr.add(bal[j, int((k+1)/2)], .5)
        expr.add(bal2[j, int((k+1)/2)], 3)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the doubles courts will be 0 if the romanian team is at least as strong
    for k in [6,7]:
        expr = LinExpr(-1*N0[k])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
        expr.add(bal[j, k-3], .5)
        expr.add(bal2[j, k-3], 3)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the singles courts will be 0 if the romanian team is at least as strong

for k in range(0,o):
    M.addConstrs(x.sum('*', j, k) <= 1 for j in range(0,m)) #all courts must have at most one player

M.update()
        
expr = LinExpr(m*sum(N0)*-1*C[2]) # subtracts overall opposing team strength from objective function
opponentSum = 0;
for k in range(0,o):
    for j in range(0,m):
        opponentSum += N0[j]
        for i in range(0,5):
            expr.add(bal[j,i], -1*C[0]) # attempt to balance as many matches as possible
            expr.addConstant(C[0])
            expr.add(bal2[j,i], -1*C[4]) # attempt to balance as many matches as possible
        for i in range(0,n):
            expr.add(x[i,j,k], N[i]*C[2]) # "overall team strength" - TODO: maybe rework condition?
            expr.add(x[i,j,k], P[i][j][k]*C[3]) # preferences
            expr.add(inds[i,j], -1*C[1]) #consecutive games penalty

M.setObjective(expr, GRB.MAXIMIZE)

M.optimize()

#Print out some reasonable output
for i in range (0,n):
    matches = ""
    for j in range (0,m):
        for k in range(0,o):
            if x[i,j,k].x == 1:
                matches += "(" + str(j) + "," + str(k) + ")"
    print("player " + str(i) + " plays at " + matches)
   
print()
for j in range (0,m):
    print("Match " + str(j + 1))
    for k in range (0,o):
        for i in range (0,n):
            if x[i,j,k].x == 1:
                print("Player " + str(i) + "(" + str(N[i]) + ") on slot " + str(k) + " against " + str(N0[k]))

Optimize a model with 344 rows, 1160 columns and 5705 nonzeros
Variable types: 0 continuous, 1160 integer (1160 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [1e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -446.0000000
Presolve removed 131 rows and 545 columns
Presolve time: 0.01s
Presolved: 213 rows, 615 columns, 2623 nonzeros
Variable types: 0 continuous, 615 integer (615 binary)

Root relaxation: objective 3.086667e+02, 437 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  308.66667    0    8 -446.00000  308.66667   169%     -    0s
H    0     0                     186.0000000  308.66667  65.9%     -    0s
H    0     0                     281.0000000  308.66667  9.85%     -    0s
H    0     0                     282.0000000  308.66667  

In [5]:
from gurobipy import *

M = Model()
# number of players in the team
n = 15
# player strength list, either 2.5 or 3.0 for each player - TODO: get actual player strengths
N = [2.5, 3.0, 2.5, 3.0, 2.5, 3.0, 3.0, 2.5, 2.5, 3.0, 3.0, 3.0, 3.0, 3.0, 2.5]
# opponent strength list, either 2.5 or 3.0 for each court position
N0 = [3.0, 3.0, 3.0, 3.0, 3.0, 2.5, 3.0, 3.0]
# number of games in the tournament
m = 8
# number of slots for a player to play in per game
o = 8
# minimum number of games each player must play
p = 2
# availability matrix. 1 if the ith player can make the jth game, 0 if they cannot
A = [[0 for j in range(m)] for i in range(n)]
A[3][0] = 1
A[12][0] = 1
A[5][0] = 1
A[10][0] = 1
A[2][0] = 1
A[4][0] = 1
A[13][0] = 1
A[6][0] = 1
A[0][0] = 1
A[8][0] = 1
A[1][0] = 1

A[12][1] = 1
A[5][1] = 1
A[8][1] = 1
A[11][1] = 1
A[10][1] = 1
A[1][1] = 1
A[13][1] = 1
A[14][1] = 1
A[2][1] = 1
A[4][1] = 1
A[0][1] = 1
A[7][1] = 1

A[9][2] = 1
A[14][2] = 1
A[8][2] = 1
A[5][2] = 1
A[0][2] = 1
A[7][2] = 1
A[1][2] = 1
A[13][2] = 1
A[3][2] = 1
A[4][2] = 1
A[2][2] = 1
A[10][2] = 1

A[1][3] = 1
A[9][3] = 1
A[12][3] = 1
A[14][3] = 1
A[4][3] = 1
A[3][3] = 1
A[13][3] = 1
A[6][3] = 1
A[2][3] = 1
A[5][3] = 1
A[10][3] = 1

A[14][4] = 1
A[12][4] = 1
A[9][4] = 1
A[11][4] = 1
A[1][4] = 1
A[4][4] = 1
A[13][4] = 1
A[6][4] = 1
A[2][4] = 1
A[0][4] = 1
A[8][4] = 1
A[7][4] = 1

A[14][5] = 1
A[12][5] = 1
A[1][5] = 1
A[5][5] = 1
A[2][5] = 1
A[7][5] = 1
A[13][5] = 1
A[6][5] = 1
A[0][5] = 1
A[11][5] = 1

A[3][6] = 1
A[12][6] = 1
A[1][6] = 1
A[9][6] = 1
A[0][6] = 1
A[10][6] = 1
A[13][6] = 1
A[6][6] = 1
A[4][6] = 1
A[5][6] = 1
A[8][6] = 1
A[7][6] = 1
A[2][6] = 1

A[1][7] = 1
A[5][7] = 1
A[11][7] = 1
A[2][7] = 1
A[4][7] = 1
A[13][7] = 1
A[6][7] = 1
# preference matrix. 1 if the ith player prefers the jth game and kth slot, -1 if they prefer not to, 0 if no preference
P = [[[0 for k in range(o)] for j in range(m)] for i in range(n)] 
for j in range(0,m):
    P[6][j][6] = 1 
    P[6][j][7] = 1 
    P[13][j][6] = 1
    P[13][j][7] = 1
# constant terms for each objective optimization, in order of: balanced games, spread schedules, high win %, player preferences,
# and heavily unbalanced (2.5/2.5 vs 3.0/3.0) matches
C = [2,.5,.5,1, 10]
# 1 if a player i plays in game j on the kth court, 0 if they do not
x = M.addVars(n, m, o, vtype=GRB.BINARY)
# 0 if the kth court for the jth game is balanced or favors the Romanian team, 1 if it disfavors the Romanian team
bal = M.addVars(m,5, vtype=GRB.BINARY)
# 0 if the kth court for the jth game is balanced or favors the Romanian team, 1 if it heavily disfavors the Romanian team
bal2 = M.addVars(m,5, vtype=GRB.BINARY)
# 0 if the ith player does not play consecutive matches starting with the jth game. 1 if they do
inds = M.addVars(n,m, vtype=GRB.BINARY)

M.update()

for i in range(0,n):
    M.addConstrs(x.sum(i, j, '*') <= A[i][j] for j in range(0,m)) #players can only play one court in a match, 
                                                                  #or none if they are not available
    M.addConstr(x.sum(i, '*', '*'), GRB.GREATER_EQUAL, p) #players must play at least p games in the season
    for j in range(0,m-1):
        M.addConstr(x.sum(i, j, '*') + x.sum(i,j+1, '*') - inds[i,j] <= 1) #ensures that inds[i,j] will be 1 if consecutive games

for j in range(0,m):
    for k in [0,2,4]:
        expr = LinExpr(-1*N0[k]-N0[k+1])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
            expr.add(x[i,j,k+1], N[i])
        expr.add(bal[j, int((k+1)/2)], .5)
        expr.add(bal2[j, int((k+1)/2)], 3)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the doubles courts will be 0 if the romanian team is at least as strong
    for k in [6,7]:
        expr = LinExpr(-1*N0[k])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
        expr.add(bal[j, k-3], .5)
        expr.add(bal2[j, k-3], 3)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the singles courts will be 0 if the romanian team is at least as strong

for k in range(0,o):
    M.addConstrs(x.sum('*', j, k) <= 1 for j in range(0,m)) #all courts must at most one player

M.update()
        
expr = LinExpr(m*sum(N0)*-1*C[2]) # subtracts overall opposing team strength from objective function
opponentSum = 0;
for k in range(0,o):
    for j in range(0,m):
        opponentSum += N0[j]
        for i in range(0,5):
            expr.add(bal[j,i], -1*C[0]) # attempt to balance as many matches as possible
            expr.addConstant(C[0])
            expr.add(bal2[j,i], -1*C[4]) # attempt to balance as many matches as possible
        for i in range(0,n):
            expr.add(x[i,j,k], N[i]*C[2]) # "overall team strength" - TODO: maybe rework condition?
            expr.add(x[i,j,k], P[i][j][k]*C[3]) # preferences
            expr.add(inds[i,j], -1*C[1]) #consecutive games penalty

M.setObjective(expr, GRB.MAXIMIZE)

M.optimize()

#Print out some reasonable output
for i in range (0,n):
    matches = ""
    for j in range (0,m):
        for k in range(0,o):
            if x[i,j,k].x == 1:
                matches += "(" + str(j) + "," + str(k) + ")"
    print("player " + str(i) + " plays at " + matches)
   
print()
for j in range (0,m):
    print("Match " + str(j + 1))
    for k in range (0,o):
        for i in range (0,n):
            if x[i,j,k].x == 1:
                print("Player " + str(i) + "(" + str(N[i]) + ") on slot " + str(k) + " against " + str(N0[k]))

Optimize a model with 344 rows, 1160 columns and 5705 nonzeros
Variable types: 0 continuous, 1160 integer (1160 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [1e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -845.2500000
Presolve removed 78 rows and 317 columns
Presolve time: 0.01s
Presolved: 266 rows, 843 columns, 3899 nonzeros
Variable types: 0 continuous, 843 integer (843 binary)

Root relaxation: objective 3.845833e+02, 1258 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  384.58333    0    5 -845.25000  384.58333   145%     -    0s
H    0     0                     371.2500000  384.58333  3.59%     -    0s
     0     0  381.91667    0   21  371.25000  381.91667  2.87%     -    0s
     0     0  381.91667    0   11  371.25000  381.91667  